In [1]:
#import libraries
import sys
import os
import time
import datetime as dt
import numpy as np
import pandas as pd
from sqlalchemy import func

sys.path.append(os.path.abspath('../../fin_data'))
from utils.date_functions import last_business_day
from utils.helper_functions import get_test_universe_tickers
from utils.postgresql_conn import get_session
from utils.postgresql_tables import Company, Tickers, HistoricalPrice
from utils.postgresql_data_query import get_effective_dates, get_company_reports_in_period

start_time = time.time()

[2024-11-13 13:58:19.992481] INFO: Norgate Data: NorgateData package v1.0.74: Init complete
connected to: dbmaster


In [2]:
## functions
def create_universe_df(session, tickers, earnings_dict):
    
    company_id_mapping = session.query(Company.ticker, Company.id).filter(Company.ticker.in_(tickers)).all()
    ticker_to_company_id = {ticker: company_id for ticker, company_id in company_id_mapping}
    
    ticker_id_mapping = session.query(Tickers.ticker, Tickers.id).filter(Tickers.ticker.in_(tickers)).all()
    ticker_to_ticker_id = {ticker: ticker_id for ticker, ticker_id in ticker_id_mapping}

    data = []
    for ticker in tickers:
        company_id = ticker_to_company_id.get(ticker)
        ticker_id = ticker_to_ticker_id.get(ticker)
        if company_id and company_id in earnings_dict:
            report_date = earnings_dict[company_id]
            data.append([ticker, company_id, ticker_id, report_date])
    
    df = pd.DataFrame(data, columns=['ticker', 'company_id', 'ticker_id','report_date'])
    
    return df

def get_price_data(session, ticker_dates_dict):
    """
    Queries the historical_price table for price data for 3 days: the most recent day before, the day of, 
    and the first available day after the report_date, if available.
    
    Args:
    - session: Database session object.
    - ticker_dates_dict: Dictionary where key is ticker_id and value is the report_date.
    
    Returns:
    - DataFrame with columns ['ticker_id', 'date', 'open', 'high', 'low', 'close', 'volume']
    """
    
    all_price_data = []
    
    for ticker_id, report_date in ticker_dates_dict.items():
        
        # Get the most recent trading day before the report_date
        day_before = (session.query(HistoricalPrice.date)
                      .filter(HistoricalPrice.ticker_id == ticker_id)
                      .filter(HistoricalPrice.date < report_date)
                      .order_by(HistoricalPrice.date.desc())
                      .first())
        
        # Get the first trading day after the report_date (extended to handle non-immediate days)
        day_after = (session.query(HistoricalPrice.date)
                     .filter(HistoricalPrice.ticker_id == ticker_id)
                     .filter(HistoricalPrice.date > report_date)
                     .order_by(HistoricalPrice.date.asc())
                     .first())
        
        # Prepare list of dates to query
        dates_to_query = [report_date]
        
        # Add day_before if it exists
        if day_before:
            dates_to_query.insert(0, day_before[0])
        
        # Add day_after if it exists
        if day_after:
            dates_to_query.append(day_after[0])
        
        # Get price data for the available days
        price_data = (session.query(HistoricalPrice.ticker_id, 
                                    HistoricalPrice.date, 
                                    HistoricalPrice.open, 
                                    HistoricalPrice.high, 
                                    HistoricalPrice.low, 
                                    HistoricalPrice.close, 
                                    HistoricalPrice.volume)
                      .filter(HistoricalPrice.ticker_id == ticker_id)
                      .filter(HistoricalPrice.date.in_(dates_to_query))
                      .all())
        
        # Extend the results to the final list
        all_price_data.extend(price_data)
    
    # Convert the result to a DataFrame
    df_price = pd.DataFrame(all_price_data, columns=['ticker_id', 'date', 'open', 'high', 'low', 'close', 'volume'])
    
    return df_price

def get_volume_stats(session, ticker_dates_dict, lookback=20):
    """
    Queries the last 'lookback' trading days before the report date and calculates volume stats (mean and std dev).
    
    Args:
    - session: Database session object.
    - ticker_dates_dict: Dictionary where key is ticker_id and value is the report_date.
    - lookback: Number of trading days to look back for volume stats (default is 20).
    
    Returns:
    - DataFrame with ticker_id, report_date, rolling_volume_mean, and rolling_volume_std.
    """
    
    all_volume_stats = []

    for ticker_id, report_date in ticker_dates_dict.items():
        # Step 1: Query volume data for the 20 trading days before the report date
        # Adding a buffer of 10 extra days to handle missing or non-trading days
        volume_data = (session.query(HistoricalPrice.ticker_id,
                                     HistoricalPrice.date,
                                     HistoricalPrice.volume)
                       .filter(HistoricalPrice.ticker_id == ticker_id)
                       .filter(HistoricalPrice.date < report_date)  # Only fetch dates before the report
                       .order_by(HistoricalPrice.date.desc())  # Most recent first
                       .limit(lookback + 10)  # Fetch extra days to handle weekends and holidays
                       .all())
        
        # Convert the result to a DataFrame
        df_volume = pd.DataFrame(volume_data, columns=['ticker_id', 'date', 'volume'])
        
        # Ensure the dates are sorted in ascending order for rolling calculations
        df_volume = df_volume.sort_values(by='date').reset_index(drop=True)
        
        # Step 2: Calculate rolling mean and std deviation of volume for the last 'lookback' days
        df_volume['rolling_volume_mean'] = df_volume['volume'].rolling(window=lookback, min_periods=1).mean()
        df_volume['rolling_volume_std'] = df_volume['volume'].rolling(window=lookback, min_periods=1).std()
        
        # Step 3: Get the last row's rolling statistics, as it represents the stats just before the report date
        last_row = df_volume.iloc[-1]
        volume_stats = {
            'ticker_id': ticker_id,
            'report_date': report_date,
            'rolling_volume_mean': last_row['rolling_volume_mean'],
            'rolling_volume_std': last_row['rolling_volume_std']
        }
        
        # Append the stats for this ticker
        all_volume_stats.append(volume_stats)
    
    # Convert the stats to a DataFrame
    df_volume_stats = pd.DataFrame(all_volume_stats)
    
    return df_volume_stats

def merge_price_data(df_universe, df_price, df_volume_stats, filter_gap=10):
    """
    Merges the price data with the universe DataFrame based on ticker_id and report_date,
    and incorporates volume stats to calculate the volume spike as a z-score.
    
    Args:
    - df_universe: DataFrame containing the tickers, company_ids, ticker_ids, and report_dates.
    - df_price: DataFrame with historical price data sorted by ticker_id and date.
    - df_volume_stats: DataFrame with rolling volume stats (mean and std dev) from get_volume_stats.
    
    Returns:
    - DataFrame with calculated gap, follow-through, and volume spike z-score.
    """
    
    # Ensure the date columns are in datetime format
    df_universe['report_date'] = pd.to_datetime(df_universe['report_date'])
    df_price['date'] = pd.to_datetime(df_price['date'])
    df_volume_stats['report_date'] = pd.to_datetime(df_volume_stats['report_date'])
    
    # Sort price data by ticker_id and date
    df_price = df_price.sort_values(by=['ticker_id', 'date']).reset_index(drop=True)
    
    # Merge price data with df_universe
    df_merged = pd.merge(df_universe, df_price, left_on=['ticker_id', 'report_date'], right_on=['ticker_id', 'date'], how='left')
    
    # Shift to get the previous day's close, next day's close, and next day's volume within each ticker_id group
    df_price['prev_close'] = df_price.groupby('ticker_id')['close'].shift(1)
    df_price['next_close'] = df_price.groupby('ticker_id')['close'].shift(-1)
    df_price['next_volume'] = df_price.groupby('ticker_id')['volume'].shift(-1)
    
    # Merge previous close and next close/volume with df_merged
    df_merged = pd.merge(df_merged, df_price[['ticker_id', 'date', 'prev_close']], 
                         left_on=['ticker_id', 'report_date'], right_on=['ticker_id', 'date'], how='left', suffixes=('', '_prev'))
    
    df_merged = pd.merge(df_merged, df_price[['ticker_id', 'date', 'next_close', 'next_volume']], 
                         left_on=['ticker_id', 'report_date'], right_on=['ticker_id', 'date'], how='left', suffixes=('', '_next'))
    
    # Drop unnecessary duplicate columns
    df_merged.drop(columns=['date', 'date_prev', 'date_next'], inplace=True)
    
    # Step 1: Merge volume statistics (rolling mean and std) with the price data
    df_merged = pd.merge(df_merged, df_volume_stats, on=['ticker_id', 'report_date'], how='left')
    
    # Step 2: Calculate gap as a percentage: (open on report day - close on prior day) / close on prior day
    df_merged['gap'] = (df_merged['open'] - df_merged['prev_close']) / df_merged['prev_close'] * 100
    
    # Step 3: Calculate follow-through as a percentage: (close on next day - open on report day) / open on report day
    df_merged['followthrough'] = (df_merged['next_close'] - df_merged['open']) / df_merged['open'] * 100
    
    # Step 4: Calculate volume spike z-score: (next_volume - rolling mean) / rolling std
    df_merged['volume_zscore'] = round((df_merged['next_volume'] - df_merged['rolling_volume_mean']) / df_merged['rolling_volume_std'], 2).astype(str) + 'σ'

    # Step 5: Convert 'gap' column to numeric for filtering and apply the filter
    df_filtered = df_merged[(df_merged['gap'].abs() + df_merged['followthrough'].abs()) >= filter_gap]  # Filter by absolute gap >= 5%

    # Step 6: Convert 'gap' and 'followthrough' back to strings with percentages using .loc[]
    df_filtered.loc[:, 'gap'] = df_filtered['gap'].round(2).astype(str) + '%'
    df_filtered.loc[:, 'followthrough'] = df_filtered['followthrough'].round(2).astype(str) + '%'

    # Step 7: Drop unnecessary columns using .loc[]
    df_filtered = df_filtered.drop(columns=['open', 'high', 'low', 'volume', 'next_volume', 
                              'rolling_volume_mean', 'rolling_volume_std']).reset_index(drop=True)
    
    # Step 8: Select the final columns to display
    df_filtered = df_filtered[['ticker', 'company_id', 'report_date', 'prev_close', 
                               'close', 'next_close', 'gap', 'followthrough', 'volume_zscore']]
    
    return df_filtered


In [3]:
## set up parameters
tickers = []
reporting_currency = None #'USD'
cols = ['Date','Open','High','Low','Close','Volume']
path = '/Users/VadimKovshov/Dropbox/INVESTMENTS/EVALUTE/STOCKS/MODEL_OUTPUTS/POWER_EARNINGS_GAP/'

w_offset = 0
d_offset = max(1, w_offset * 7)
end = last_business_day(offset=d_offset)
start = last_business_day(offset=d_offset + 28)
current_date = dt.date.today()
print(f'Historical range: {end.strftime("%Y-%m-%d")} to {start.strftime("%Y-%m-%d")}')

Historical range: 2024-11-12 to 2024-10-15


In [4]:
# get universe dates, tickers & id's
session = get_session()
eff_date, pr_date = get_effective_dates(offset_0=w_offset)
print(f'Universe dates: {eff_date.strftime("%Y-%m-%d")}, {pr_date.strftime("%Y-%m-%d")}')

for date in [eff_date, pr_date]:
    tickers.extend(get_test_universe_tickers(session, date, currency_reporting=reporting_currency))
tickers = list(set(tickers))
print(f'Universe tickers: {len(tickers)}')

connected to: dbmaster
Universe dates: 2024-11-08, 2024-11-01
Universe tickers: 1760


In [ ]:
company_reports = get_company_reports_in_period(session, start_date=start, end_date=end, dimension='arq')
df_universe = create_universe_df(session, tickers=tickers, earnings_dict=company_reports) \
                                .sort_values('report_date', ascending=False) \
                                .reset_index(drop=True)
ticker_dates_dict = dict(zip(df_universe['ticker_id'], df_universe['report_date']))
print(f'Companies reported: {len(df_universe)}')

In [ ]:
# obtain & merge price data with universe data, volume data and calculate the gap and follow-through
df_price = get_price_data(session, ticker_dates_dict=ticker_dates_dict)
df_volume_stats = get_volume_stats(session, ticker_dates_dict=ticker_dates_dict)

In [ ]:
df_merged = merge_price_data(df_universe=df_universe, df_price=df_price, df_volume_stats=df_volume_stats, filter_gap=10)
df_merged.to_clipboard(index=False)

In [ ]:
df_merged.to_csv(f'{path}power_earnings_gap_{current_date}.csv', index=False)
df_merged

,ticker,company_id,report_date,prev_close,close,next_close,gap,followthrough,volume_zscore
0,BAH,1533,2024-10-25,166.55,181.930,186.00,8.68%,2.76%,2.21σ
1,SAIA,12085,2024-10-25,414.43,461.650,479.38,6.12%,9.0%,1.62σ
2,GME,14835,2024-10-25,20.63,20.570,22.73,0.1%,10.07%,4.53σ
3,MHK,12766,2024-10-25,151.87,130.880,131.71,-11.17%,-2.36%,4.72σ
4,BCPC,11466,2024-10-25,166.16,171.460,170.79,9.61%,-6.23%,4.72σ
5,NWL,7524,2024-10-25,7.18,8.730,9.53,20.89%,9.79%,3.74σ
6,CRI,11847,2024-10-25,65.69,56.950,56.39,-2.57%,-11.89%,4.85σ
7,QS,6817,2024-10-25,6.50,6.190,5.69,0.78%,-13.14%,0.89σ
8,CNC,10376,2024-10-25,61.62,64.220,61.65,13.6%,-11.93%,0.81σ
9,TNET,4149,2024-10-25,91.63,80.310,82.65,-10.89%,1.22%,6.58σ
